# Transacciones y locks

## ¿Que es una transacción?

Una **transacción** es una secuencia de varias (1 o más) **operaciónes** que  modifican o cunsultan la base de datos

* Transferencias de dinero entre cuentas
* Compra por internet
* Registrar un curso

![image](img/transacciones.png)

## Acid

* Atomicity: O se ejecutN todas la operaciones de la transacción o no se eectua ninguna 
* ConsistenyÑ cada transacción prserva la cosnirencia la bd (restreciones de integridad, etc)
* isolation: cada transaccion debe ejecutarse como si estuviese ejecutand sola de forma aislada
* Durability: los cambios que hace cada transaccion son permantes en el tiempo independiento de cualquier tipo de fallla.

**Transaction Manager** se sencarga de asegurar Isolation y Consistency

**Log y Recovery Manager** se encargan de asegurar
Atomicity y Durability


Esto es porque las transacciones se hacen simultanemtene y deben ser en serie para que no se pierdan datos, si no por ejemplo, en una transaccin doble una cuent LLE LA PLATA ORIGINAL 1000 y le deposita 100 queda en 1100, y el sgeundo lee el dinero original antes de la transaccion 1000 y le dona 200 queda en 1200 final, cuando deberia quedar en 1300 si fuese en serie.

## Que pasa si no hay atomicity y durability?

Se corta la uz y la transaccion se queda a la mitad

Se corta la luz cuando la transaccione estaba a la mitad, la base de dato vuelve a su estado pero perdemos una mitad

un cambio heco en trasaccio no se ve reflejado en la base de datos

## Sin consistency

La bd viola restriciones momentanemante

Al ejecturar la transacion queda la bd que no cumple con las estricciones

## Sin isolatio

EL sitstema de la bd planifica el orden de operaciones

puede pasar lo descrito antes,no se reali **en serie**

## Sintaxis de transacciones

```sql
START TRANSACTION

UPDATE cuentas
SET saldo = saldo - v
WHERE cid = 1

UPDATE cuentas
SET saldo = saldo + v
WHERE cid = 2
COMMIT
```

START TRANSACTION y COMMIT nos permiten
agrupar operaciones en una sola transacción



## Operaciones d ela transaccion

R(X): Leer el valor de X

W(X): Escribir el valor de X

A: Abortar la transacción

C: Commit

## Conflictos con transacciones

### Dirty read (Write - Read)

ocurren caundo una transaccion accede a datos que **han sido modificados** por otra trasaccion que aun no se ha terminado

### Non-repeatable read (Read - Write)

Ocurren cuando una filas es leida dos veces y el alor **cambia entre ambas lecturas** esot se debe a aque otra transaccion actualiza o modifica la fela entre las dos leturas individuales

### Phantom read (Write - Write)

Ocurren cuando dos transaccion que intentan cacrualizar la misma fila son procesadas en un tiempo tal que una de las actualizaciones **sobrescriba a la otra** (esribe una luego la otra)




## Ejemplo phantom read (Write - Write)

“Una transacción sobreescribe los datos que otra
tx ya había escrito”


* T2: R(X) = True (asiento 24 libre)
* T1: R(X) = True (asiento 24 libre)
* T2: W(X) = Reserve el asiento 24
* T1: C
* T2: W(X) = Reserve el asiento 24 (estaba ocupado pero lo reservo igual)


## Ejemplo Dirty Read (Write - Read)

“Una tx. lee lo que otra tx escribió pero no se había
confirmado aún.”

* T2: W(X) = Reserve el asiento 24
* T1: R(X) = False (no quedan asientos)
* T1: W(X) = Reserve primera clase
* T1: C
* T2: A (como no lo toma, entonces si esta libre el asiento 24)

## Ejemplo Non-repeatable read (Read - Write)

“Una tx. sobreescribe un dato que otra ya había leído antes
pero no había confirmado.”

* T1: R(X) = True (quedan asientos)
* T2: W(X) = Reserve el asiento 24
* T2: C
* T1: R(X) = True (quedan asientos, pero son menos)



## Schedules

## ¿Que es un schedule?

Un **schedule S** es una secuencia de operaciones primitivas de **una o mas transacciones**, tal que para toda transacon, las acciones de ella aparencen **en eljmismo orden que en su definicion**.

**schedule S es Serial si no hay INTERCALACION ENTRE LAS ACCIONES**

## Ejemplo de schedule

* T1: R(X), W(X), R(Y), W(Y)
* T2: R(Y), W(Y), R(X), W(X)

## Ejemplo de schedule serial

* T1: R(X), W(X)
* T2: ------------R(Y), W(Y)



## Schedule serializable

Significa que se puede transformar en un scheduyle serial sin cambiar el resultado

Para que sea serializable el orden original no tiene que afectar el resultado

## Ejemplo de schedule serializable

![image](img/serializable.png)

## Ejemplo de schedule no serializable

![image](img/no_serializable.png)

## Acciones NO conflictivas

Las siguientes accion no son cofilictivbasc para dos tranasciones disitntas i y j

* Ri(X) y Rj(Y)
* Ri(X) y Wj(Y) con X != Y
* Wi(X) y Rj(Y) con X != Y
* Wi(X) y Wj(Y) con X != Y

Luego podemos cambiarle el orden en una **schedule**

**Schedule es confilict serializable** si puedo transiofrmalo a sertial usando permutyaciones


**Schedule conflict serializable** implica que es serializable,
pero hay schedules que son serializables y no son conflict serializables

## Strict 2PL

Es el protyocolo de concurrencia mas usado en los dbms

Esta basado en uso de locks

2 reglas:

Regla 1: SI una transaccion quiere **leer/modificar** un objeto primero pide un **shared lock / exclusive lock**

Ningun otro puede pedir el lock mientra lo usa

Regla 2: Cuando la transaccion **se completa** **libera todos los locks que tenia**



## Transacicones en SQL


```sql
START TRANSACTION
    SELECT a_nombre
    FROM Actores;
COMMIT;
```

Para desacer una transaccion se usa ROLLBACK

```sql
START TRANSACTION
    SELECT a_nombre
    FROM Actores;
ROLLBACK;

## SQL en Nivel 


SET TRANSACTION ISOLATION LEVEL Level READ
ONLY

SET TRANSACTION ISOLATION LEVEL Level READ
WRITE

LEVEL: SERIALIZABLE, REPEATABLE READ,
READ COMMITTED, READ UNCOMMITTED

| Nivel de aislamiento | Dirty Read (WR) | Unrepeatable Read (RW)| Phantom Read (WW) |
|----------------------|------------|-------------------|--------------|
| SERIALIZABLE         | No         | No                | No           |
| REPEATABLE READ      | No         | No                | Maybe        |
| READ COMMITTED       | No         | Maybe             | Maybe        |
| READ UNCOMMITTED     | Maybe      | Maybe             | Maybe        |


Default:

SET TRANSACTION ISOLATION LEVEL SERIALIZABLE
READ WRITE
